In [16]:
import importlib
import ifimage_tools
importlib.reload(ifimage_tools)
import pandas as pd
import warnings

In [19]:
import os
import numpy as np
from stardist.matching import matching
image_dir = "Reorgnized Ground Truth"
masks_dir = "Reorgnized Ground Truth/mask"
dataset = ifimage_tools.IfImageDataset(image_dir, masks_dir, {})
dataset.load_data()

old_version_sample_ids = ["6390", "8408", "8406", "8405v2", "8405", "8407"]
for sample_id in old_version_sample_ids:
    if sample_id in dataset.samples:
        del dataset.samples[sample_id]

In [1]:
from preprocessing import SamplePreprocessor,batch_process_zip_rois,batch_rename

In [2]:
batch_process_zip_rois(['/Users/macbookair/Downloads/new524'])

正在处理ZIP文件: /Users/macbookair/Downloads/new524/NeuN_Scribbled/RoiSet_1116_NeuN_CellBodies.zip
ROI '0050-0094.roi' 以 label 1 绘制。
ROI '0015-0219.roi' 以 label 2 绘制。
ROI '0049-0469.roi' 以 label 3 绘制。
ROI '0250-0024.roi' 以 label 4 绘制。
ROI '0423-0111.roi' 以 label 5 绘制。
ROI '0445-0238.roi' 以 label 6 绘制。
ROI '0587-0167.roi' 以 label 7 绘制。
ROI '0622-0012.roi' 以 label 8 绘制。
ROI '0302-0491.roi' 以 label 9 绘制。
ROI '0395-0469.roi' 以 label 10 绘制。
ROI '0112-0517.roi' 以 label 11 绘制。
ROI '0181-0705.roi' 以 label 12 绘制。
ROI '0220-0789.roi' 以 label 13 绘制。
ROI '0011-0833.roi' 以 label 14 绘制。
ROI '0005-0902.roi' 以 label 15 绘制。
ROI '0107-0881.roi' 以 label 16 绘制。
ROI '0113-0975.roi' 以 label 17 绘制。
ROI '0095-1030.roi' 以 label 18 绘制。
ROI '0107-1096.roi' 以 label 19 绘制。
ROI '0177-1062.roi' 以 label 20 绘制。
ROI '0194-1155.roi' 以 label 21 绘制。
ROI '0231-0956.roi' 以 label 22 绘制。
ROI '0303-1167.roi' 以 label 23 绘制。
ROI '0324-1120.roi' 以 label 24 绘制。
ROI '0351-1087.roi' 以 label 25 绘制。
ROI '0382-1067.roi' 以 label 26 绘制。
ROI '0

In [3]:
batch_rename('/Users/macbookair/Downloads/new524', 
             '/Users/macbookair/Downloads/new524/renamed')

✅ (generic) RoiSet_1111_NeuN_CellBodies.npy → neun_1111/neun_1111_cellbodies.npy
✅ (generic) RoiSet_1108_NeuN_CellBodies.npy → neun_1108/neun_1108_cellbodies.npy
✅ (generic) RoiSet_1110_NeuN_CellBodies.npy → neun_1110/neun_1110_cellbodies.npy
✅ (generic) RoiSet_1109_NeuN_CellBodies.npy → neun_1109/neun_1109_cellbodies.npy
✅ (generic) RoiSet_1113_NeuN_CellBodies.npy → neun_1113/neun_1113_cellbodies.npy
✅ (generic) RoiSet_1112_NeuN_CellBodies.npy → neun_1112/neun_1112_cellbodies.npy
✅ (generic) RoiSet_1114_NeuN_CellBodies.npy → neun_1114/neun_1114_cellbodies.npy
✅ (generic) RoiSet_1115_NeuN_CellBodies.npy → neun_1115/neun_1115_cellbodies.npy
✅ (generic) RoiSet_1116_NeuN_CellBodies.npy → neun_1116/neun_1116_cellbodies.npy
✅ (generic) RoiSet_1116_NeuN_CellBodies.zip → neun_1116/neun_1116_cellbodies.zip
✅ (generic) RoiSet_1115_NeuN_CellBodies.zip → neun_1115/neun_1115_cellbodies.zip
✅ (generic) RoiSet_1114_NeuN_CellBodies.zip → neun_1114/neun_1114_cellbodies.zip
✅ (generic) RoiSet_1112_NeuN

In [4]:
import os
import shutil

def collect_ground_truth(src_root, dest_dir, exts=(".npy", ".tiff", ".tif")):
    """
    Recursively copy all files ending with exts from src_root into dest_dir.
    """
    os.makedirs(dest_dir, exist_ok=True)
    for root, _, files in os.walk(src_root):
        for f in files:
            if f.lower().endswith(exts):
                src_path = os.path.join(root, f)
                dst_path = os.path.join(dest_dir, f)
                # 如果你担心重名被覆盖，可以改成：
                # rel = os.path.relpath(root, src_root).replace(os.sep, "_")
                # dst_path = os.path.join(dest_dir, f"{rel}_{f}")
                shutil.copy2(src_path, dst_path)
                print(f"📂 Copied {src_path} → {dst_path}")

if __name__ == "__main__":
    src = "/Users/macbookair/Downloads/new524/renamed"
    dst = "/Users/macbookair/PycharmProjects/ifimage/Reorgnized Ground Truth"
    collect_ground_truth(src, dst)
    print("🎉 All done, babe!")


📂 Copied /Users/macbookair/Downloads/new524/renamed/neun_1111/neun_1111_cellbodies.npy → /Users/macbookair/PycharmProjects/ifimage/Reorgnized Ground Truth/neun_1111_cellbodies.npy
📂 Copied /Users/macbookair/Downloads/new524/renamed/neun_1116/neun_1116_cellbodies.npy → /Users/macbookair/PycharmProjects/ifimage/Reorgnized Ground Truth/neun_1116_cellbodies.npy
📂 Copied /Users/macbookair/Downloads/new524/renamed/neun_1110/neun_1110_cellbodies.npy → /Users/macbookair/PycharmProjects/ifimage/Reorgnized Ground Truth/neun_1110_cellbodies.npy
📂 Copied /Users/macbookair/Downloads/new524/renamed/neun_1109/neun_1109_cellbodies.npy → /Users/macbookair/PycharmProjects/ifimage/Reorgnized Ground Truth/neun_1109_cellbodies.npy
📂 Copied /Users/macbookair/Downloads/new524/renamed/neun_1108/neun_1108_cellbodies.npy → /Users/macbookair/PycharmProjects/ifimage/Reorgnized Ground Truth/neun_1108_cellbodies.npy
📂 Copied /Users/macbookair/Downloads/new524/renamed/neun_1115/neun_1115_cellbodies.npy → /Users/macb

In [12]:
import zipfile
import os

# 1) List your sample IDs
sample_ids = ['1116','1111','1110','1108','1115','1112','1113','1114','1109']
zip_path = '/Users/macbookair/Downloads/NeuN.zip'
out_dir = 'extracted_tiffs'

os.makedirs(out_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zf:
    # Build a lookup for each sample
    results = {sid: {'dapi': None, 'marker': None} for sid in sample_ids}

    # 2) Scan all TIFFs once
    for info in zf.infolist():
        if info.is_dir() or not info.filename.lower().endswith('.tiff'):
            continue
        fname = os.path.basename(info.filename).lower()

        # Check each sample ID
        for sid in sample_ids:
            if sid in info.filename:
                if 'b0c0' in fname and results[sid]['dapi'] is None:
                    results[sid]['dapi'] = info
                elif 'b0c1' in fname and results[sid]['marker'] is None:
                    results[sid]['marker'] = info

    # 3) Extract the streams we found
    for sid, files in results.items():
        dapi_info   = files['dapi']
        marker_info = files['marker']

        if not dapi_info or not marker_info:
            print(f"[!] Sample {sid}:",
                  "DAPI missing"   if not dapi_info   else "",
                  "Marker missing" if not marker_info else "")
            continue

        # Write out
        with zf.open(dapi_info) as ds, open(os.path.join(out_dir, f"neun_{sid}.tiff"), 'wb') as out:
            out.write(ds.read())
        with zf.open(marker_info) as ms, open(os.path.join(out_dir, f"neun_{sid}_marker.tiff"), 'wb') as out:
            out.write(ms.read())

        print(f"✓ Extracted neun_{sid}.tiff and neun_{sid}_marker.tiff")


✓ Extracted neun_1116.tiff and neun_1116_marker.tiff
✓ Extracted neun_1111.tiff and neun_1111_marker.tiff
✓ Extracted neun_1110.tiff and neun_1110_marker.tiff
✓ Extracted neun_1108.tiff and neun_1108_marker.tiff
✓ Extracted neun_1115.tiff and neun_1115_marker.tiff
✓ Extracted neun_1112.tiff and neun_1112_marker.tiff
✓ Extracted neun_1113.tiff and neun_1113_marker.tiff
✓ Extracted neun_1114.tiff and neun_1114_marker.tiff
✓ Extracted neun_1109.tiff and neun_1109_marker.tiff


In [9]:
zf.namelist()

['NeuN/',
 'NeuN/10101327/',
 'NeuN/10101327/Gray/',
 'NeuN/10101327/Gray/Snap-1108.tiff',
 'NeuN/10101327/Gray/Snap-1108.tiff_files/',
 'NeuN/10101327/Gray/Snap-1108.tiff_files/Snap-1108_b0c0x0-1388y0-1040.tiff_metadata.xml',
 'NeuN/10101327/Gray/Snap-1108.tiff_files/Snap-1108_b0c1x0-1388y0-1040.tiff_metadata.xml',
 'NeuN/10101327/Gray/Snap-1108.tiff_files/Snap-1108_b0c2x0-1388y0-1040.tiff',
 'NeuN/10101327/Gray/Snap-1108.tiff_files/Snap-1108_b0c2x0-1388y0-1040.tiff_metadata.xml',
 'NeuN/10101327/Gray/Snap-1108.tiff_files/Snap-1108_dapi-b0c0x0-1388y0-1040.tiff',
 'NeuN/10101327/Gray/Snap-1108.tiff_files/Snap-1108_NeuN-b0c1x0-1388y0-1040.tiff',
 'NeuN/10101327/Gray/Snap-1108.tiff_metadata.xml',
 'NeuN/10101327/Gray/Snap-1109.tiff',
 'NeuN/10101327/Gray/Snap-1109.tiff_files/',
 'NeuN/10101327/Gray/Snap-1109.tiff_files/Snap-1109_b0c0x0-1388y0-1040.tiff_metadata.xml',
 'NeuN/10101327/Gray/Snap-1109.tiff_files/Snap-1109_b0c1x0-1388y0-1040.tiff_metadata.xml',
 'NeuN/10101327/Gray/Snap-1109.